<a href="https://colab.research.google.com/github/nse-yu/colab-ML/blob/main/human_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 人の顔識別

### データロード＆前処理編

> 画像セットの読み込み

In [2]:
!cp "/content/drive/MyDrive/imgs_deep.zip" .

In [3]:
!unzip imgs_deep.zip

Archive:  imgs_deep.zip
replace imgs_deep/Aaron_Eckhart_0001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

> 読み込み

In [4]:
import matplotlib.pyplot as plt
import os
import cv2
import glob
import numpy as np
from PIL import Image

> 画像読み込み関数

In [55]:
def imget(paths):
  """
  画像のpathリストを指定し、その画像をリストで返す。
  """

  #データセットを格納する変数
  x_train = []
  #リサイズ後のサイズ
  IMG_RATE = 0.4

  #個別のFilePathに対して処理していきます。
  for p in paths:
    pathEach = p
    print(pathEach)

    img_before = cv2.imread(pathEach)
    print(img_before.shape)
    img_after = cv2.resize(img_before, dsize=(32,32))
    img_after = cv2.cvtColor(img_after,cv2.COLOR_BGR2RGB)
    img_after = img_after.astype("float64") / 255

    x_train.append(img_after)
  return np.array(x_train)

> 人の画像

In [ ]:
os.chdir("/content/imgs_deep") #階層をカレントからimgs_deepに移動
paths = glob.glob("*") #階層内のすべてのファイルパスを取得

x_train_true = imget(paths)

> その他の画像

In [6]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [7]:
data = unpickle("/content/data_batch_1")

In [62]:
x_train_false = data[b"data"]
print(x_train_false.shape) #10000個の画像、3072のうち最初の1/3はred,次がgreen,最後がblue

#red
x_train_false_r = x_train_false[:,:1024].reshape(10000,32,32)

#green
x_train_false_g = x_train_false[:,1024:2048].reshape(10000,32,32)

#blue
x_train_false_b = x_train_false[:,2048:].reshape(10000,32,32)

#rgb化
x_train_false = np.array([x_train_false_r,x_train_false_g,x_train_false_b]).transpose((1,2,3,0)).astype("float64") / 255

(10000, 3072)


> 比較

In [63]:
print(x_train_true.max())
print(x_train_false.max())

1.0
1.0


> ラベルの付与

In [87]:
#trueラベル
x_t = np.zeros(shape=(x_train_true.shape[0]))
x_t[:] = True

#falseラベル
x_f = np.zeros(shape=(x_train_false.shape[0]))
x_f[:] = False

x_train = np.concatenate([x_train_true,x_train_false])
t_train = np.concatenate([x_t,x_f])

> 訓練データの結合

In [88]:
r_idx = np.random.permutation(len(x_train)) #indexをランダムにすることで、データとラベルの対応を保持することができる

#この２つが訓練用データとなる
x_train = x_train[r_idx]
t_train = t_train[r_idx]

> ランダム画像の確認

In [ ]:
for entry in x_train[:4]:
  plt.imshow(entry)
  plt.show()
print(t_train[:4])

### 学習編

In [91]:
from tensorflow import keras
import numpy as np

> モデルひな形作成

In [ ]:
model = keras.layers.Input(shape=())